In [18]:
import pandas as pd
import matplotlib.pyplot as plt
from ast import literal_eval
import numpy as np
import nltk
from nltk.corpus import stopwords
# nltk.download('stopwords')
from time import time
import importlib
import math

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.preprocessing import normalize
from scipy.sparse import csr_matrix, vstack
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

import tensorflow as tf

from tensorflow import keras
from keras import preprocessing

from keras.layers import Dense
from keras.models import Sequential, load_model

from keras.models import Model
from keras_preprocessing.sequence import pad_sequences
from keras_preprocessing.text import Tokenizer

from keras_preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, GlobalAveragePooling1D

import transformers as ppb # pytorch-pretrained-bert
# from transformers import __version__; print(__version__)

from transformers import BertTokenizer
from transformers import BertModel, BertForMaskedLM

import torch

import pipeline as pp
import models as ml


4.27.3


In [4]:
importlib.reload(pp)
from_file = "../datasets/big/dataset_bin.csv"
BATCH_SIZE = 5000

pp.get_dataframe_with_distribution(from_file, BATCH_SIZE, [0.8,0.1,0.1], [False, False, False], 
                                   out_file="../datasets/sample/dataset_unbalanced.csv", get_frame=False)

100%|██████████| 200000/200000 [00:00<00:00, 805975.39it/s]


entries read: 200000


In [3]:
importlib.reload(pp)

def Clean_data(file, new_file):
    stopwords_lst = stopwords.words('english')
    pp.apply_pipeline(file, [
            # Clean content
            (pp.Clean_data(), 'content'),
            (pp.Tokenizer(), "content"),
            (pp.Remove_stopwords(stopwords_lst), "content"),
            (pp.Stem(), "content"),
            (pp.Combine_Content(), "content", "content_combined"),
            # Clean authors
            (pp.Clean_author(), "authors"),
            # Clean title
            (pp.Clean_data(), 'title'),
            (pp.Tokenizer(), "title"),
            (pp.Remove_stopwords(stopwords_lst), "title"),
            (pp.Stem(), "title"),
            (pp.Combine_Content(), "title"),
            # Clean domain
            (pp.Clean_domain(), 'domain'),
            # Combine columns (used as features)
            (pp.Join_str_columns(["content_combined", "authors"]), None, "content_authors"),
            (pp.Join_str_columns(["content_combined", "title"]), None, "content_title"),
            (pp.Join_str_columns(["content_combined", "domain"]), None, "content_domain"),
            (pp.Join_str_columns(["content_combined", "domain", "authors", "title"]), None, "content_domain_authors_title")
        ],
        new_file=new_file,
        progress_bar=True,
    )

Clean_data("../datasets/sample/dataset_unbalanced.csv", "../datasets/sample/dataset_unbalanced_cleaned.csv")

100%|██████████| 5000/5000 [00:00<00:00, 24996.27it/s]


finish time: 50.90818643569946


In [1]:
def split_csr_data(data: pd.DataFrame, feature: str, y, stack_func, set="set", get_val=True):
    train = data[data[set] == 0]
    val = data[data[set] == 1]
    test = data[data[set] == 2]
    X_train, y_train = stack_func(train[feature]), train[y].astype(int)
    X_val, y_val = stack_func(val[feature]), val[y].astype(int)
    X_test, y_test = stack_func(test[feature]), test[y].astype(int)
    if not get_val:
        return X_train, X_test, y_train, y_test
    return X_train, X_val, X_test, y_train, y_val, y_test

NameError: name 'pd' is not defined

# Model A

In [12]:
df = pd.read_csv("../datasets/sample/dataset_unbalanced.csv")

In [16]:
X = df['content'].values
y = df['type_binary'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

In [18]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [22]:
max_len = 500
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

In [25]:
model = tf.keras.Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=max_len),
    LSTM(units=64, dropout=0.2, recurrent_dropout=0.2),
    Dense(units=1, activation='sigmoid')
])

In [28]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pad, y_train, batch_size=32, epochs=10, validation_data=(X_test_pad, y_test))

Epoch 1/10
125/125 [==============================] - 35s 257ms/step - loss: 0.5595 - accuracy: 0.7103 - val_loss: 0.4562 - val_accuracy: 0.7710
Epoch 2/10
125/125 [==============================] - 34s 271ms/step - loss: 0.3328 - accuracy: 0.8580 - val_loss: 0.4303 - val_accuracy: 0.8040
Epoch 3/10
125/125 [==============================] - 32s 260ms/step - loss: 0.1814 - accuracy: 0.9283 - val_loss: 0.5382 - val_accuracy: 0.7870
Epoch 4/10
125/125 [==============================] - 34s 271ms/step - loss: 0.0855 - accuracy: 0.9728 - val_loss: 0.6164 - val_accuracy: 0.7870
Epoch 5/10
125/125 [==============================] - 36s 290ms/step - loss: 0.0363 - accuracy: 0.9905 - val_loss: 0.8345 - val_accuracy: 0.7890
Epoch 6/10
125/125 [==============================] - 38s 308ms/step - loss: 0.0619 - accuracy: 0.9790 - val_loss: 0.6924 - val_accuracy: 0.7640
Epoch 7/10
125/125 [==============================] - 36s 291ms/step - loss: 0.0588 - accuracy: 0.9785 - val_loss: 0.8416 - val_ac

In [ ]:
loss, accuracy = model.evaluate(X_test_pad, y_test)
print("Accuracy: {:.2f}%".format(accuracy * 100))

# Model B - tensorflow (word embedding, neural network)

In [5]:
# Split data into training and testing sets
df = pd.read_csv("../datasets/sample/dataset_unbalanced_cleaned.csv")
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")

# content cleaned
tokenizer.fit_on_texts(df[df["set"] == 0]["content"])
sequences = tokenizer.texts_to_sequences(df["content"])
df["padded_sequences"] = pad_sequences(sequences, maxlen=1000, truncating="post").tolist()

# all cleaned
tokenizer.fit_on_texts(df[df["set"] == 0]["content_domain_authors_title"])
sequences = tokenizer.texts_to_sequences(df["content_domain_authors_title"])
df["padded_sequences_all"] = pad_sequences(sequences, maxlen=1000, truncating="post").tolist()

In [19]:
# Define the model architecture
models = [
    (tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=10000, output_dim=16, input_length=1000),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ]), "Model-A"),
    # Results: [0.4473037123680115, 0.777999997138977]
    # Results: [0.4245592951774597, 0.7940000295639038]
    (tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=10000, output_dim=16, input_length=1000), # 10000 words, 64 dimensions
        tf.keras.layers.LSTM(units=16, dropout=0.2, recurrent_dropout=0.2),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ]), "Model-B")
    # Results: [0.6940779685974121, 0.7940000295639038]
    # Results: [0.43015122413635254, 0.871999979019165]
]
# Compile the model
for model, name in models:
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

for feature in ["padded_sequences", "padded_sequences_all"]:
    print("Training model with feature: {}".format(feature))
    X_train, X_val, X_test, y_train, y_val, y_test = split_csr_data(df, feature, "type_binary", lambda x: np.array(x.tolist()), get_val=True)

    for model, name in models:
        print(f"Training {name}")
        # Train the model
        model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

        # Evaluate the model
        print(f"Results:", model.evaluate(X_test, y_test))


Training model with feature: padded_sequences
Training Model-A
Epoch 1/10
125/125 [==============================] - 2s 8ms/step - loss: 0.6889 - accuracy: 0.5792 - val_loss: 0.6874 - val_accuracy: 0.5420
Epoch 2/10
125/125 [==============================] - 1s 7ms/step - loss: 0.6645 - accuracy: 0.6300 - val_loss: 0.6511 - val_accuracy: 0.6300
Epoch 3/10
125/125 [==============================] - 1s 6ms/step - loss: 0.6163 - accuracy: 0.6710 - val_loss: 0.6043 - val_accuracy: 0.6880
Epoch 4/10
125/125 [==============================] - 1s 7ms/step - loss: 0.5545 - accuracy: 0.7360 - val_loss: 0.5525 - val_accuracy: 0.7420
Epoch 5/10
125/125 [==============================] - 1s 7ms/step - loss: 0.4922 - accuracy: 0.7803 - val_loss: 0.5175 - val_accuracy: 0.7600
Epoch 6/10
125/125 [==============================] - 1s 7ms/step - loss: 0.4436 - accuracy: 0.8062 - val_loss: 0.4885 - val_accuracy: 0.7760
Epoch 7/10
125/125 [==============================] - 1s 7ms/step - loss: 0.4092 - ac

In [87]:
# Define the model architecture
model = tf.keras.Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=max_len),
    LSTM(units=64, dropout=0.2, recurrent_dropout=0.2),
    Dense(units=1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(X_train, y_train, epochs=30, validation_data=(X_val, y_val))

# Evaluate the model
model.evaluate(X_test, y_test)



# [1.0781421661376953, 0.800000011920929]

Epoch 1/30
125/125 [==============================] - 2s 8ms/step - loss: 0.6741 - accuracy: 0.5870 - val_loss: 0.6326 - val_accuracy: 0.6420
Epoch 2/30
125/125 [==============================] - 1s 7ms/step - loss: 0.5415 - accuracy: 0.7070 - val_loss: 0.5130 - val_accuracy: 0.7020
Epoch 3/30
125/125 [==============================] - 1s 8ms/step - loss: 0.3882 - accuracy: 0.8250 - val_loss: 0.4208 - val_accuracy: 0.8220
Epoch 4/30
125/125 [==============================] - 1s 8ms/step - loss: 0.2817 - accuracy: 0.8867 - val_loss: 0.4082 - val_accuracy: 0.8060
Epoch 5/30
125/125 [==============================] - 1s 7ms/step - loss: 0.2077 - accuracy: 0.9160 - val_loss: 0.4322 - val_accuracy: 0.8240
Epoch 6/30
125/125 [==============================] - 1s 7ms/step - loss: 0.1561 - accuracy: 0.9400 - val_loss: 0.4569 - val_accuracy: 0.8160
Epoch 7/30
125/125 [==============================] - 1s 7ms/step - loss: 0.1199 - accuracy: 0.9517 - val_loss: 0.4958 - val_accuracy: 0.8280
Epoch 

[1.2360297441482544, 0.8019999861717224]

In [93]:
pdf1 = pd.read_csv("../datasets/sample/dataset_unbalanced.csv")
pdf2 = pp.apply_pipeline_pd_tqdm(pdf1, [(pp.Tokenizer(), "content", "tokenized")])

100%|██████████| 5000/5000 [00:00<00:00, 15719.50it/s]


In [94]:
list = []
_ = pdf2["tokenized"].apply(lambda x: list.extend(x))

In [96]:
list

['To',
 'the',
 'Editor:',
 'Re',
 "''Drop",
 'Out',
 'of',
 'the',
 "College''",
 '(editorial,',
 'March',
 '14):',
 'Having',
 'written',
 'in',
 'favor',
 'of',
 'Electoral',
 'College',
 'reform',
 'since',
 '2000,',
 'I,',
 'too,',
 'would',
 'prefer',
 'the',
 'popular',
 'election',
 'of',
 'the',
 'president.',
 'But',
 'the',
 'gimmick',
 'you',
 'endorse,',
 'of',
 'having',
 'individual',
 'states',
 'bind',
 'their',
 'electors',
 'to',
 'vote',
 'for',
 'the',
 'national',
 'popular-vote',
 'winner,',
 'seems',
 'problematic',
 'for',
 'one',
 'basic',
 'reason.',
 'What',
 'is',
 'to',
 'stop',
 'state',
 'legislatures',
 'with',
 'strong',
 'partisan',
 'loyalties',
 'of',
 'their',
 'own',
 'from',
 'abandoning',
 'such',
 'an',
 'agreement',
 'when',
 'urgent',
 'calculations',
 'of',
 'party',
 'advantage',
 'come',
 'to',
 'the',
 'fore?',
 'What',
 'one',
 'legislature',
 'can',
 'do,',
 'another',
 'can',
 'undo.',
 'In',
 'the',
 'end,',
 'difficult',
 'as',
 'it'

In [88]:
["hej", "med", "dig"] + ["hej", "med", "dig"]

['hej', 'med', 'dig', 'hej', 'med', 'dig']

# Model C - tensorflow (word embedding, neural network)

### Model D - tensorflow (word embedding, neural network)

# ...

In [7]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten

# Create a corpus of text data
corpus = ["I like apples", "I love oranges", "She hates pears", "He dislikes bananas"]

# Create a dictionary of words and their indices
word_dict = {}
for sentence in corpus:
    for word in sentence.split():
        if word not in word_dict:
            word_dict[word] = len(word_dict)

print(word_dict)

# Convert the corpus into a matrix of word indices
corpus_matrix = np.zeros((len(corpus), len(word_dict)))
for i, sentence in enumerate(corpus):
    for word in sentence.split():
        corpus_matrix[i, word_dict[word]] = 1

print(corpus_matrix)

# Create a neural network model with an embedding layer
model = Sequential()
model.add(Embedding(len(word_dict), 10, input_length=len(word_dict)))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(corpus_matrix, np.array([1, 1, 0, 0]), epochs=100, verbose=0)

# Get the learned word embeddings
embeddings = model.get_weights()[0]

# Print the learned embeddings for each word
for word, index in word_dict.items():
    print(word, embeddings[index])


{'I': 0, 'like': 1, 'apples': 2, 'love': 3, 'oranges': 4, 'She': 5, 'hates': 6, 'pears': 7, 'He': 8, 'dislikes': 9, 'bananas': 10}
[[1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]]
I [ 0.06881227 -0.13503245  0.13067168  0.16277544  0.1175928   0.14362365
 -0.05564805  0.05248439  0.08392346 -0.15577602]
like [-0.14159463  0.15149534 -0.14801809 -0.1548039  -0.07514571 -0.1058769
  0.05275943 -0.07659105 -0.15096149  0.1543007 ]
apples [-0.0305975  -0.04509356  0.03028085  0.01999826 -0.00269739 -0.00310541
  0.02410186 -0.04725834  0.02400244  0.04329363]
love [-0.04896925 -0.00796487 -0.04935968 -0.01651973 -0.0330081  -0.00728096
  0.00932461  0.00094824  0.04020016  0.00554812]
oranges [-0.01422012 -0.01638101 -0.03042551  0.00591964 -0.01169596  0.04766793
 -0.02327204 -0.00236449  0.01989326  0.02445111]
She [-0.00154535 -0.03032771  0.00611943 -0.0191129  -0.04995989 -0.01201131
 -0.03

In [ ]:
# Load the dataset
df = pd.read_csv('fake_news_dataset.csv')

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Convert the text data to sequences of word indices
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad the sequences to a fixed length
max_len = 1000
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

# Create the word embedding matrix
word_index = tokenizer.word_index
embedding_dim = 100
embedding_matrix = np.zeros((len(word_index)+1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = word_embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Create the neural network model
model = Sequential()
model.add(Embedding(len(word_index)+1, embedding_dim, input_length=max_len, weights=[embedding_matrix], trainable=False))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_pad, y_train, batch_size=32, epochs=10, validation_data=(X_test_pad, y_test))

# Evaluate the model on the test set
score = model.evaluate(X_test_pad, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
